![](http://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/UPM/EscPolitecnica/EscUpmPolit_p.gif "UPM")

# Trabajo final SITC
## Análisis de sentimientos en Twitter

Departamento de Ingeniería de Sistemas Telemáticos. Universidad Politécnica de Madrid.

Realizado por:
- Juan Bermudo Mera
- Margarita Bolívar Jiménez
- Lourdes Fernández Nieto
- Ramón Pérez Hernández

© 2017

# Preprocesado del Corpus y del fichero de tweets

## Tabla de contenidos

* [Procesado del Corpus anotado TASS](#1.-Procesado-del-Corpus-anotado-TASS)
	* [Conversión del fichero XML a dataframe](#Conversión-del-fichero-XML-a-dataframe)
    * [Procesado del dataframe](#Procesado-del-dataframe)
    * [Binarización del sentimiento](#Binarización-del-sentimiento)
* [Procesado de los tweets con geolocalización](#2.-Procesado-de-los-tweets-con-geolocalización)


In [1]:
# Se importan todas las librerías necesarias (Algunas necesitan instalación con: pip install <nombre_paquete>)

# Detección del lenguaje 
import langid
from langdetect import detect
import textblob

# Procesado del fichero xml
import xml.etree.ElementTree as ET
from lxml import etree
import pandas as pd
import re
import numpy as np

## 1. Procesado del Corpus anotado TASS

* ### Conversión del fichero XML a dataframe

In [2]:
# Lectura del fichero xml con el corpus anotado TASS
xml_data = 'ficheros/Raw/general-tweets-train-tagged.xml'

# Conversión del fichero xml a un dataframe
def xml2df(xml_data):
    parser = ET.XMLParser(encoding='utf-8')
    tree = ET.parse(xml_data, parser)
    root = tree.getroot()
    all_records = []
    headers = ['tweetid','user','content','date','lang','topic','polarity_value','polarity_type']

    for tweet in root.iter('tweet'):
        record = []
        record.append(tweet.find('tweetid').text)
        record.append(tweet.find('user').text)
        record.append(tweet.find('content').text)
        record.append(tweet.find('date').text)
        record.append(tweet.find('lang').text)
        
        t = ""
        for topics in tweet.iter('topics'):
            for topic in topics.iter('topic'):
                t = t+" "+topic.text
        record.append(t)

        for sentiments in tweet.iter('sentiments'):
            #pol_ent = ""
            pol_val = ""
            pol_type = ""
            for polarity in sentiments.iter('polarity'):
                #pol_ent = pol_ent+" "+polarity.find('entity').text
                pol_val = pol_val+" "+polarity.find('value').text
                pol_type = pol_type+" "+polarity.find('type').text
            #record.append(pol_ent)
            record.append(pol_val)
            record.append(pol_type)
                
        all_records.append(record)
    return pd.DataFrame(all_records, columns=headers)

tweets_df = xml2df(xml_data)

In [3]:
tweets_df.head()

,tweetid,user,content,date,lang,topic,polarity_value,polarity_type
0,142389495503925248,ccifuentes,"\n Salgo de #VeoTV , que día más largoooooo... \n",2011-12-02T00:47:55,es,otros,NONE,AGREEMENT
1,142389933619945473,CarmendelRiego,\n\n@PauladeLasHeras No te libraras de ayudar ...,2011-12-02T00:49:40,es,otros,NEU NEU,DISAGREEMENT DISAGREEMENT
2,142391947707940864,CarmendelRiego,\n @marodriguezb Gracias MAR \n,2011-12-02T00:57:40,es,otros,P P,AGREEMENT AGREEMENT
3,142416095012339712,mgilguerrero,"\n\nOff pensando en el regalito Sinde, la que ...",2011-12-02T02:33:37,es,política economía,N+ N+ N+,AGREEMENT AGREEMENT AGREEMENT
4,142422495721562112,paurubio,\n\nConozco a alguien q es adicto al drama! Ja...,2011-12-02T02:59:03,es,otros,P+,AGREEMENT


In [4]:
tweets_df.shape

(7219, 8)

Se observa que hay 7219 tweets

In [5]:
tweets_df.dtypes

tweetid           object
user              object
content           object
date              object
lang              object
topic             object
polarity_value    object
polarity_type     object
dtype: object

* ### Procesado del dataframe

In [6]:
# Nos quedamos solo con las columnas que nos interesan para el análisis de sentimiento
tweets_df = tweets_df[['content','polarity_value','polarity_type']]

In [7]:
# Los campos polarity_value y polarity_type pueden tener más de un valor (ya que el primero es el general, 
# y el resto están referidos a entidades que aparecen en el texto). Por eso, nos quedaremos solo con el 
# primer valor que es el general

for i in range(0,tweets_df.shape[0]):
    tweets_df.loc[i,'polarity_value'] = tweets_df.loc[i,'polarity_value'].split()[0] 
    tweets_df.loc[i,'polarity_type'] = tweets_df.loc[i,'polarity_type'].split()[0] 

In [8]:
# Se eliminan el salto de línea (caracter '\n'), los espacios en los extremos del tweet, las URLs y los emojis
car=['\n']

def borrar_caracteres(texto):
    for caracter in car:
        texto=texto.replace(caracter,"")
    return texto

emoji_pattern = re.compile(u'['
     u'\U0001F300-\U0001F64F'
     u'\U0001F680-\U0001F6FF'
     u'\u2600-\u26FF\u2700-\u27BF]+', 
     re.UNICODE)

for i in range(0,tweets_df.shape[0]):
    tweets_df.content[i]=borrar_caracteres(tweets_df.content[i])
    tweets_df.content[i]=tweets_df.content[i].strip()
    tweets_df.content[i]=re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweets_df.content[i])
    tweets_df.content[i]=emoji_pattern.sub(r'', tweets_df.content[i])

In [9]:
# Nos quedamos con aquellos tweets que tienen polaridad asignada y aquellos en los que hay acuerdo 
# (agreement) en el sentimiento presente en el tweet
tweets_df = tweets_df.query('polarity_type != "DISAGREEMENT" and polarity_value != "NONE"')

In [10]:
# Se comprueba que se han eliminado correctamente los links (reseteamos los índices para usar los correctos)
tweets_df = tweets_df.reset_index(drop=True)
tw_with_links = tweets_df.content.str.contains('http.*$')
tw_with_links.value_counts(normalize=False)

False    5012
True        1
Name: content, dtype: int64

In [11]:
# Se consultará qué tweet es el que tiene enlace, supuestamente.
np.where(tw_with_links == True)

(array([1359]),)

In [12]:
tweets_df.loc[[1359]].content.tolist()

['RT @elviswarrior: Con el carro de Manolo Escobar? “@mariviromero: ¿Dónde fue a parar el dinero de los ERE?  - http:/ ...']

In [13]:
# Este tweet se modificará manualmente
tweets_df.set_value(1359,'content',"RT @elviswarrior: Con el carro de Manolo Escobar? “@mariviromero: ¿Dónde fue a parar el dinero de los ERE?")
tweets_df.loc[[1359]].content.tolist()

['RT @elviswarrior: Con el carro de Manolo Escobar? “@mariviromero: ¿Dónde fue a parar el dinero de los ERE?']

In [14]:
# Ahora, ya no hay tweets con enlaces
tw_with_links = tweets_df.content.str.contains('http.*$')
tw_with_links.value_counts(normalize=False)

False    5013
Name: content, dtype: int64

In [15]:
tweets_df.head()

,content,polarity_value,polarity_type
0,@marodriguezb Gracias MAR,P,AGREEMENT
1,"Off pensando en el regalito Sinde, la que se v...",N+,AGREEMENT
2,Conozco a alguien q es adicto al drama! Ja ja ...,P+,AGREEMENT
3,Toca @crackoviadeTV3 . Grabación dl especial N...,P+,AGREEMENT
4,Buen día todos! Lo primero mandar un abrazo gr...,P+,AGREEMENT


In [16]:
tweets_df.shape

(5013, 3)

* ### Binarización del sentimiento

In [17]:
# Se eliminan los tweets con polaridad neutra
tweets_df = tweets_df[tweets_df.polarity_value != 'NEU']

# Se añade una nueva columna con los valores de polaridad 1 si ésta es positiva ó 0 si ésta es negativa
tweets_df['polarity_bin'] = 0
tweets_df.polarity_bin[tweets_df.polarity_value.isin(['P', 'P+'])] = 1
tweets_df.polarity_bin.value_counts(normalize=True)

/Users/monphdez/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1    0.577447
0    0.422553
Name: polarity_bin, dtype: float64

In [18]:
tweets_df.shape

(4700, 4)

In [19]:
tweets_df.head()

,content,polarity_value,polarity_type,polarity_bin
0,@marodriguezb Gracias MAR,P,AGREEMENT,1
1,"Off pensando en el regalito Sinde, la que se v...",N+,AGREEMENT,0
2,Conozco a alguien q es adicto al drama! Ja ja ...,P+,AGREEMENT,1
3,Toca @crackoviadeTV3 . Grabación dl especial N...,P+,AGREEMENT,1
4,Buen día todos! Lo primero mandar un abrazo gr...,P+,AGREEMENT,1


In [20]:
# Se exportan los tweets preprocesados a un fichero Excel
tweets_df[['content','polarity_bin']].to_excel('ficheros/Preprocesados/tweets_corpus_header.xlsx', header=True, index=False)

## 2. Procesado de los tweets con geolocalización

In [21]:
# Lectura del fichero de tweets
tweets = pd.read_excel('ficheros/Raw/spain_geocode.xlsx', header=0, encoding='iso8859_15')
tweets.shape

(205789, 19)

In [22]:
tweets.head()

,Tweet Id,Date,Hour,User Name,Nickname,Bio,Tweet content,Favs,RTs,Latitude,Longitude,Country,Place (as appears on Bio),Profile picture,Followers,Following,Listed,Tweet language (ISO 639-1),Tweet Url
0,723496204018040836,2016-04-22,12:58,Andoni Merino,AndoniMerino,NaN,San Francisco será la primera gran ciudad en e...,NaN,NaN,43.26271,-2.92528,ES,Bilbao,http://pbs.twimg.com/profile_images/1669281616...,60.0,283.0,5.0,es,http://www.twitter.com/AndoniMerino/status/723...
1,723496203875430400,2016-04-22,12:58,antonio silva,antoniosilvatt,NaN,https://t.co/LDpaxOi2C2,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/6321183806...,3.0,16.0,NaN,es,http://www.twitter.com/antoniosilvatt/status/7...
2,723496203518881792,2016-04-22,12:58,Ayuntamiento de Rota,RotaAyto,Twitter oficial del Ayuntamiento de Rota. Sígu...,@jjaviruiz preside el acto de homenaje a los f...,NaN,NaN,36.62545,-6.36220,ES,"Rota, Cádiz",http://pbs.twimg.com/profile_images/6901508342...,2169.0,52.0,34.0,es,http://www.twitter.com/RotaAyto/status/7234962...
3,723496203493756928,2016-04-22,12:58,Carlos F. Valledor,cfv1961,Todos somos muy ignorantes. Lo que ocurre es q...,"Regalo para socios: 'Talento a la fuga', el li...",NaN,NaN,40.41600,-3.70300,ES,"Madrid, España",http://pbs.twimg.com/profile_images/7145772157...,375.0,319.0,94.0,es,http://www.twitter.com/cfv1961/status/72349620...
4,723496203464413184,2016-04-22,12:58,Showman,TheShowmanEU,Estoy en YouTube por dinero•Twitch:https://t.c...,@muzikplumz 😘,NaN,NaN,40.41600,-3.70300,ES,"Madrid, Comunidad de Madrid",http://pbs.twimg.com/profile_images/6783568295...,262.0,241.0,3.0,es,http://www.twitter.com/TheShowmanEU/status/723...


In [23]:
# Nos quedamos solo con las columnas que nos interesan
tweets = tweets[['User Name','Tweet content','Latitude','Longitude','Tweet language (ISO 639-1)']]

In [24]:
# Cambiamos el nombre de algunas columnas
tweets.rename(columns={'User Name':'user', 'Tweet content':'content','Tweet language (ISO 639-1)':'lan_excel'}, inplace = True) 

In [25]:
# Filtramos los tweets y solo nos quedamos solo con los de la Comunidad de Madrid
min_lon = -4.585707
max_lon = -3.036635
min_lat = 39.865278
max_lat = 41.192925

tweets = tweets[(tweets.Latitude.notnull()) & (tweets.Longitude.notnull())]

tweets = tweets[(tweets.Longitude > min_lon) & (tweets.Longitude < max_lon) & (tweets.Latitude > min_lat) & (tweets.Latitude < max_lat)]
tweets.shape

(40516, 5)

In [26]:
# Filtramos los tweets y nos quedamos solo con aquellos que estén en español
tweets = tweets[(tweets.lan_excel == 'es')]
tweets.shape

(33204, 5)

In [27]:
# Funciones para detectar el lenguaje y comprobar que, efectivamente, todos están en español
def langid_safe(tweet):
    try:
        return langid.classify(tweet)[0]
    except Exception as e:
        pass
        
def langdetect_safe(tweet):
    try:
        return detect(tweet)
    except Exception as e:
        pass

def textblob_safe(tweet):
    try:
        return textblob.TextBlob(tweet).detect_language()
    except Exception as e:
        pass

In [28]:
# Detectamos los tweets en español (sólo se usará una función para reducir el tiempo de ejecución)

tweets['lang_langid'] = tweets.content.apply(langid_safe)
#tweets['lang_langdetect'] = tweets.content.apply(langdetect_safe)
#tweets['lang_textblob'] = tweets.content.apply(textblob_safe)

In [29]:
# Filtramos los tweets y nos quedamos solo con aquellos que estén en español
#tweets = tweets.query(''' lang_langdetect == 'es' or lang_langid == 'es' or lang_textblob == 'es'  ''')
tweets = tweets[(tweets.lang_langid == 'es')]
tweets.shape

(26363, 6)

In [30]:
tweets.head()

,user,content,Latitude,Longitude,lan_excel,lang_langid
3,Carlos F. Valledor,"Regalo para socios: 'Talento a la fuga', el li...",40.416,-3.703,es,es
5,Miguel Ángel Valero,"La recuperación del ‘ladrillo’ dispara el 144,...",40.416,-3.703,es,es
6,Carlos F. Valledor,TELEVISIÓN SERIES - Cuenta atrás para conocer ...,40.416,-3.703,es,es
22,Centro Tangram,Nuestra taza de hoy es un homenaje a uno de lo...,40.416,-3.703,es,es
25,↯ ↯ ↯,@MCadepe nos vemos el 19 de mayooo!!!😍😍😍😘😘😘,40.416,-3.703,es,es


In [31]:
# Descartamos las columnas del idioma
tweets = tweets[['user','content','Latitude','Longitude']]

In [32]:
# Se eliminan las URLs y los emoji de los tweets
for i in range(0,tweets.shape[0]):
    x = tweets.index[i]
    tweets.content[x]=re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweets.content[x])
    # El patrón de emojis se usó en el Corpus
    tweets.content[x]=emoji_pattern.sub(r'', tweets.content[x])
        
tweets.shape

/Users/monphdez/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/monphdez/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(26363, 4)

In [33]:
tweets.head()

,user,content,Latitude,Longitude
3,Carlos F. Valledor,"Regalo para socios: 'Talento a la fuga', el li...",40.416,-3.703
5,Miguel Ángel Valero,"La recuperación del ‘ladrillo’ dispara el 144,...",40.416,-3.703
6,Carlos F. Valledor,TELEVISIÓN SERIES - Cuenta atrás para conocer ...,40.416,-3.703
22,Centro Tangram,Nuestra taza de hoy es un homenaje a uno de lo...,40.416,-3.703
25,↯ ↯ ↯,@MCadepe nos vemos el 19 de mayooo!!!,40.416,-3.703


In [34]:
# Se comprueba que se han eliminado correctamente los links (reseteamos los índices para 
# usar los correctos)
tweets = tweets.reset_index(drop=True)
tw_with_links = tweets.content.str.contains('http.*$')
tw_with_links.value_counts(normalize=False)

False    26360
True         3
Name: content, dtype: int64

In [35]:
# Se consultará qué tweets son los que tienen enlace, supuestamente.
np.where(tw_with_links == True)

(array([ 4143, 11710, 18874]),)

In [36]:
# El primero y el tercero tienen enlace (el segundo es de un hashtag)
print(tweets.loc[[4143]].content.tolist())
print(tweets.loc[[11710]].content.tolist())
print(tweets.loc[[18874]].content.tolist())

['Si tienes una boda este fin de semana...No te lo pienses más. \nhttp: //www.asos.com/es/Top-de-tirante ... ']
['#elpríncipefinalhttp es ahora una tendencia en Spain ']
['Una bonita mañana celebrando el día mundial del circo. MADPAC.https//vimeo.com/163159793']


In [37]:
# Estos tweets se modificarán manualmente
tweets.set_value(4143,'content',"Si tienes una boda este fin de semana...No te lo pienses más.")
tweets.set_value(18874,'content',"Una bonita mañana celebrando el día mundial del circo. MADPAC.")
print(tweets.loc[[4143]].content.tolist())
print(tweets.loc[[18874]].content.tolist())

['Si tienes una boda este fin de semana...No te lo pienses más.']
['Una bonita mañana celebrando el día mundial del circo. MADPAC.']


In [38]:
# Ahora, sólo queda 1 tweet que cumple la condición, pero se vio que no tenía enlace, así que terminado
tw_with_links = tweets.content.str.contains('http.*$')
tw_with_links.value_counts(normalize=False)

False    26362
True         1
Name: content, dtype: int64

In [39]:
# Se exportan los tweets preprocesados a un fichero Excel
tweets[['content','Latitude','Longitude']].to_excel('ficheros/Preprocesados/tweets_spainGeo.xlsx', header=True, index=False)

<hr>

## Licencia

El notebook está licenciado libremente bajo la licencia [Creative Commons Attribution Share-Alike](https://creativecommons.org/licenses/by/2.0/).

La base del código empleado procede del trabajo de Manuel Garrido llamado [Cómo hacer Análisis de Sentimiento en español](http://pybonacci.org/2015/11/24/como-hacer-analisis-de-sentimiento-en-espanol-2/).

© 2017 - Juan Bermudo Mera, Margarita Bolívar Jiménez, Lourdes Fernández Nieto, Ramón Pérez Hernández.

Universidad Politécnica de Madrid.